In [ ]:
# mount drive
from google.colab import drive

drive.mount("/content/drive")

In [ ]:
import os

# which data to use, normalized images or not
NORM = True

# get data, might take a while
if not NORM:
    if os.path.exists("/content/drive/MyDrive/NCT-CRC-HE-100K-NONORM.zip?download=1"):
        !unzip '/content/drive/MyDrive/NCT-CRC-HE-100K-NONORM.zip?download=1'
    else:
        print("Data not found, downloading...")
        !wget https://zenodo.org/record/1214456/files/NCT-CRC-HE-100K-NONORM.zip\?download\=1
        !unzip '/content/NCT-CRC-HE-100K-NONORM.zip?download=1'
else:
    if os.path.exists("/content/drive/MyDrive/NCT-CRC-HE-100K.zip?download=1"):
        !unzip '/content/drive/MyDrive/NCT-CRC-HE-100K.zip?download=1'
    else:
        print("Data not found, downloading...")
        !wget https://zenodo.org/record/1214456/files/NCT-CRC-HE-100K.zip\?download\=1
        !unzip '/content/NCT-CRC-HE-100K.zip?download=1'

In [ ]:
import os
import shutil
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import (
    ImageDataGenerator,
    load_img,
    img_to_array,
)
from PIL import Image

MORE = False
# Set the path to the NCT-CRC-HE-100K dataset folder
if not NORM:
    dataset_path = "/content/NCT-CRC-HE-100K-NONORM"
else:
    dataset_path = "/content/NCT-CRC-HE-100K"

# Set the path to the folder where you want to save the train and test data
train_test_path = "/content/train_test_folder"

# Set the train-test split ratio
train_ratio = 0.8

# Create train and test folders
train_path = os.path.join(train_test_path, "train")
test_path = os.path.join(train_test_path, "test")
os.makedirs(train_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)

if not MORE:
    # Loop through each class folder in the dataset
    for class_folder in os.listdir(dataset_path):
        class_folder_path = os.path.join(dataset_path, class_folder)

        # Get the list of images in the class folder
        images_list = os.listdir(class_folder_path)

        # Shuffle the images randomly
        random.shuffle(images_list)

        # Split the images into train and test based on the train_ratio
        train_images_list = images_list[: int(len(images_list) * train_ratio)]
        test_images_list = images_list[int(len(images_list) * train_ratio) :]

        # Create a separate directory for each class within the train and test directories
        train_class_path = os.path.join(train_path, class_folder)
        test_class_path = os.path.join(test_path, class_folder)
        os.makedirs(train_class_path, exist_ok=True)
        os.makedirs(test_class_path, exist_ok=True)

        # Copy the train images to the train class directory
        for train_image in train_images_list:
            train_image_path = os.path.join(class_folder_path, train_image)
            train_image_dest_path = os.path.join(train_class_path, train_image)
            shutil.copy(train_image_path, train_image_dest_path)

        # Copy the test images to the test class directory
        for test_image in test_images_list:
            test_image_path = os.path.join(class_folder_path, test_image)
            test_image_dest_path = os.path.join(test_class_path, test_image)
            shutil.copy(test_image_path, test_image_dest_path)
else:
    train_datagen = ImageDataGenerator(
        rotation_range=25,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode="reflect",
    )
    train_aug_factor = 1
    for class_folder in os.listdir(dataset_path):
        class_folder_path = os.path.join(dataset_path, class_folder)
        images_list = os.listdir(class_folder_path)
        random.shuffle(images_list)
        train_images_list = images_list[: int(len(images_list) * train_ratio)]
        test_images_list = images_list[int(len(images_list) * train_ratio) :]

        train_class_path = os.path.join(train_path, class_folder)
        test_class_path = os.path.join(test_path, class_folder)
        os.makedirs(train_class_path, exist_ok=True)
        os.makedirs(test_class_path, exist_ok=True)

        # Copy original train images to train_aug directory
        for train_image in train_images_list:
            train_image_path = os.path.join(class_folder_path, train_image)
            train_image_dest_path = os.path.join(train_class_path, train_image)
            shutil.copy(train_image_path, train_image_dest_path)

        # Generate and copy augmented train images to train_aug directory
        for train_image in train_images_list:
            train_image_path = os.path.join(class_folder_path, train_image)
            train_image_name, train_image_ext = os.path.splitext(
                train_image
            )  # Split filename and extension
            for i in range(train_aug_factor):
                train_image_copy_path = os.path.join(
                    train_class_path, f"{train_image_name}_copy_{i}{train_image_ext}"
                )  # Add extension to new filename
                shutil.copy(train_image_path, train_image_copy_path)
                train_image_copy = img_to_array(load_img(train_image_copy_path))
                train_image_copy = train_datagen.random_transform(train_image_copy)
                train_image_copy = train_image_copy.astype("uint8")
                train_image_copy = Image.fromarray(train_image_copy)
                #    print(np.max(train_image_copy), np.min(train_image_copy), type(train_image_copy), np.array(train_image_copy).shape)
                train_image_copy.save(train_image_copy_path)

        # Copy original test images to test directory
        for test_image in test_images_list:
            test_image_path = os.path.join(class_folder_path, test_image)
            test_image_dest_path = os.path.join(test_class_path, test_image)
            shutil.copy(test_image_path, test_image_dest_path)

In [ ]:
import numpy as np
# Calculate class weights
num_classes = len(os.listdir(train_path))
train_class_counts = {}
for class_folder in os.listdir(train_path):
    class_path = os.path.join(train_path, class_folder)
    train_class_counts[class_folder] = len(os.listdir(class_path))

train_class_weights_dict = {}
for idx, class_folder in enumerate(os.listdir(train_path)):
    class_weight = sum(train_class_counts.values()) / (
        num_classes * train_class_counts[class_folder]
    )
    train_class_weights_dict[idx] = class_weight

print(train_class_weights_dict)
train_class_weights = list(train_class_weights_dict.values())
train_class_weights = np.array(train_class_weights) / sum(train_class_weights)
print(train_class_weights)

In [ ]:
import numpy as np
import cv2
import albumentations as A
import os
import tensorflow as tf

# data generator following tensorflow.keras.utils.Sequence 
class CustomDataGenerator(tf.keras.utils.Sequence):
    def __init__(
        self,
        directory,
        batch_size,
        target_size=(256, 256),
        shuffle=True,
        augmentations=None,
    ):
        self.directory = directory
        self.batch_size = batch_size
        self.target_size = target_size # (height, width)
        self.shuffle = shuffle
        self.augmentations = augmentations
        self.class_names = sorted(os.listdir(directory))
        self.num_classes = len(self.class_names)
        self.samples = []
        for i, class_name in enumerate(self.class_names):
            class_dir = os.path.join(self.directory, class_name)
            for filename in os.listdir(class_dir):
                self.samples.append((os.path.join(class_dir, filename), i))
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.samples) / float(self.batch_size)))

    # get and preprocess data
    def __getitem__(self, idx):
        batch_samples = self.samples[
            idx * self.batch_size : (idx + 1) * self.batch_size
        ]
        batch_images = []
        batch_labels = []
        for sample in batch_samples:
            image = cv2.imread(sample[0])
            # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, self.target_size)
            label = sample[1]
            if self.augmentations is not None:
                augmented = self.augmentations(image=image)
                image = augmented["image"]
            batch_images.append(image)
            batch_labels.append(label)
        return np.array(batch_images) / 255.0, tf.keras.utils.to_categorical(
            batch_labels, num_classes=self.num_classes
        )

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.samples)

# data augmentation using albumentations if needed
train_augmentations = A.Compose(
    [
        A.HorizontalFlip(p=0.5),
        A.Rotate(limit=(-20, 20), p=0.5),
        A.RandomBrightnessContrast(p=0.25, brightness_limit=0.2, contrast_limit=0.2),
        A.ColorJitter(p=0.2),
        A.Blur(p=0.2, blur_limit=(3, 7)),
        A.RandomRotate90(p=0.4),
        A.VerticalFlip(p=0.5),
        A.OneOf(
            [
                A.ShiftScaleRotate(
                    shift_limit=0.2, scale_limit=0.2, rotate_limit=20, p=0.5
                ),
                A.ShiftScaleRotate(
                    shift_limit=0.18,
                    scale_limit=0.18,
                    rotate_limit=18,
                    border_mode=cv2.BORDER_CONSTANT,
                    value=0,
                    interpolation=cv2.INTER_NEAREST,
                    p=0.15,
                ),
            ],
            p=0.4,
        ),
        # A.CoarseDropout(max_holes=2, max_height=18, max_width=18, min_holes=1, p=0.1),
    ]
)

# create generators
train_generator = CustomDataGenerator(
    directory=train_path,
    batch_size=32,
    target_size=(256, 256),
    shuffle=True,
    augmentations=train_augmentations,
)

test_generator = CustomDataGenerator(
    directory=test_path,
    batch_size=32,
    target_size=(256, 256),
    shuffle=False,
    augmentations=None,
)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# get a batch of images and labels from the generator
batch_images, batch_labels = train_generator.__getitem__(0)
batch_images_1, batch_labels_1 = test_generator.__getitem__(0)

# display images and labels
def dis_gen(x, y):
    n = len(x)
    fig, axs = plt.subplots(nrows=1, ncols=n, figsize=(30, 30))
    for i in range(n):
        axs[i].imshow(x[i])
        axs[i].set_title(f"Label: {y[i]}")
        axs[i].axis(False)
    plt.show()


def check(x, y):
    print(f"shapes: x: {x.shape}, y: {y.shape}")
    print(f"norms: x: {np.min(x), np.max(x)}, y: {np.min(y), np.max(y)}")
    print(f"types: x: {type(x)}, y; {type(y)}")


dis_gen(batch_images, batch_labels)
dis_gen(batch_images_1, batch_labels_1)
check(batch_images, batch_labels)
check(batch_images_1, batch_labels_1)

In [ ]:
"""
Creates a EfficientNetV2 Model as defined in:
Mingxing Tan, Quoc V. Le. (2021).
EfficientNetV2: Smaller Models and Faster Training
arXiv preprint arXiv:2104.00298.
"""

"""
Implementation by leondgarse:
https://github.com/leondgarse
changed some activation functions 
"""
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Activation,
    Add,
    BatchNormalization,
    Conv2D,
    Dense,
    DepthwiseConv2D,
    Dropout,
    GlobalAveragePooling2D,
    Input,
    PReLU,
    Reshape,
    Multiply,
    GroupNormalization,
    MultiHeadAttention,
)

from tensorflow.keras.utils import get_custom_objects

# mish activation function has shown better performance than other activation functions in many cases
# replacing activation functions with mish activation functions improves performance
class Mish(Activation):
    """
    Mish Activation Function.
    .. math::
        mish(x) = x * tanh(softplus(x)) = x * tanh(ln(1 + e^{x}))
    Shape:
        - Input: Arbitrary. Use the keyword argument `input_shape`
        (tuple of integers, does not include the samples axis)
        when using this layer as the first layer in a model.
        - Output: Same shape as the input.
    Examples:
        >>> X = Activation('Mish', name="conv1_act")(X_input)
    """

    def __init__(self, activation, **kwargs):
        super(Mish, self).__init__(activation, **kwargs)
        self.__name__ = "Mish"


def mish(inputs):
    return inputs * tf.math.tanh(tf.math.softplus(inputs))


get_custom_objects().update({"Mish": Mish(mish)})

BATCH_NORM_DECAY = 0.9
BATCH_NORM_EPSILON = 0.001
TORCH_BATCH_NORM_EPSILON = 1e-5
CONV_KERNEL_INITIALIZER = keras.initializers.VarianceScaling(
    scale=2.0, mode="fan_out", distribution="truncated_normal"
)
# CONV_KERNEL_INITIALIZER = 'glorot_uniform'

BLOCK_CONFIGS = {
    "b0": {  # width 1.0, depth 1.0
        "first_conv_filter": 32,
        "expands": [1, 4, 4, 4, 6, 6],
        "out_channels": [16, 32, 48, 96, 112, 192],
        "depthes": [1, 2, 2, 3, 5, 8],
        "strides": [1, 2, 2, 2, 1, 2],
        "use_ses": [0, 0, 0, 1, 1, 1],
        "rescale_mode": "torch",
    },
    "b1": {  # width 1.0, depth 1.1
        "first_conv_filter": 32,
        "expands": [1, 4, 4, 4, 6, 6],
        "out_channels": [16, 32, 48, 96, 112, 192],
        "depthes": [2, 3, 3, 4, 6, 9],
        "strides": [1, 2, 2, 2, 1, 2],
        "use_ses": [0, 0, 0, 1, 1, 1],
        "rescale_mode": "torch",
    },
    "b2": {  # width 1.1, depth 1.2
        "first_conv_filter": 32,
        "output_conv_filter": 1408,
        "expands": [1, 4, 4, 4, 6, 6],
        "out_channels": [16, 32, 56, 104, 120, 208],
        "depthes": [2, 3, 3, 4, 6, 10],
        "strides": [1, 2, 2, 2, 1, 2],
        "use_ses": [0, 0, 0, 1, 1, 1],
        "rescale_mode": "torch",
    },
    "b3": {  # width 1.2, depth 1.4
        "first_conv_filter": 40,
        "output_conv_filter": 1536,
        "expands": [1, 4, 4, 4, 6, 6],
        "out_channels": [16, 40, 56, 112, 136, 232],
        "depthes": [2, 3, 3, 5, 7, 12],
        "strides": [1, 2, 2, 2, 1, 2],
        "use_ses": [0, 0, 0, 1, 1, 1],
        "rescale_mode": "torch",
    },
    "t": {  # width 1.4 * 0.8, depth 1.8 * 0.9, from timm
        "first_conv_filter": 24,
        "output_conv_filter": 1024,
        "expands": [1, 4, 4, 4, 6, 6],
        "out_channels": [24, 40, 48, 104, 128, 208],
        "depthes": [2, 4, 4, 6, 9, 14],
        "strides": [1, 2, 2, 2, 1, 2],
        "use_ses": [0, 0, 0, 1, 1, 1],
        "rescale_mode": "torch",
    },
    "s": {  # width 1.4, depth 1.8
        "first_conv_filter": 24,
        "output_conv_filter": 1280,
        "expands": [1, 4, 4, 4, 6, 6],
        "out_channels": [24, 48, 64, 128, 160, 256],
        "depthes": [2, 4, 4, 6, 9, 15],
        "strides": [1, 2, 2, 2, 1, 2],
        "use_ses": [0, 0, 0, 1, 1, 1],
        "rescale_mode": "tf",
    },
    "early": {  # S model discribed in paper early version https://arxiv.org/pdf/2104.00298v2.pdf
        "first_conv_filter": 24,
        "output_conv_filter": 1792,
        "expands": [1, 4, 4, 4, 6, 6],
        "out_channels": [24, 48, 64, 128, 160, 272],
        "depthes": [2, 4, 4, 6, 9, 15],
        "strides": [1, 2, 2, 2, 1, 2],
        "use_ses": [0, 0, 0, 1, 1, 1],
        "rescale_mode": "tf",
    },
    "m": {  # width 1.6, depth 2.2
        "first_conv_filter": 24,
        "output_conv_filter": 1280,
        "expands": [1, 4, 4, 4, 6, 6, 6],
        "out_channels": [24, 48, 80, 160, 176, 304, 512],
        "depthes": [3, 5, 5, 7, 14, 18, 5],
        "strides": [1, 2, 2, 2, 1, 2, 1],
        "use_ses": [0, 0, 0, 1, 1, 1, 1],
        "rescale_mode": "tf",
    },
    "l": {  # width 2.0, depth 3.1
        "first_conv_filter": 32,
        "output_conv_filter": 1280,
        "expands": [1, 4, 4, 4, 6, 6, 6],
        "out_channels": [32, 64, 96, 192, 224, 384, 640],
        "depthes": [4, 7, 7, 10, 19, 25, 7],
        "strides": [1, 2, 2, 2, 1, 2, 1],
        "use_ses": [0, 0, 0, 1, 1, 1, 1],
        "rescale_mode": "tf",
    },
    "xl": {
        "first_conv_filter": 32,
        "output_conv_filter": 1280,
        "expands": [1, 4, 4, 4, 6, 6, 6],
        "out_channels": [32, 64, 96, 192, 256, 512, 640],
        "depthes": [4, 8, 8, 16, 24, 32, 8],
        "strides": [1, 2, 2, 2, 1, 2, 1],
        "use_ses": [0, 0, 0, 1, 1, 1, 1],
        "rescale_mode": "tf",
    },
}

FILE_HASH_DICT = {
    "b0": {
        "21k-ft1k": "4e4da4eb629897e4d6271e131039fe75",
        "21k": "5dbb4252df24b931e74cdd94d150f25a",
        "imagenet": "9abdc43cb00f4cb06a8bdae881f412d6",
    },
    "b1": {
        "21k-ft1k": "5f1aee82209f4f0f20bd24460270564e",
        "21k": "a50ae65b50ceff7f5283be2f4506d2c2",
        "imagenet": "5d4223b59ff268828d5112a1630e234e",
    },
    "b2": {
        "21k-ft1k": "ec384b84441ddf6419938d1e5a0cbef2",
        "21k": "9f718a8bbb7b63c5313916c5e504790d",
        "imagenet": "1814bc08d4bb7a5e0ed3ccfe1cf18650",
    },
    "b3": {
        "21k-ft1k": "4a27827b0b2df508bed31ae231003bb1",
        "21k": "ade5bdbbdf1d54c4561aa41511525855",
        "imagenet": "cda85b8494c7ec5a68dffb335a254bab",
    },
    "l": {
        "21k-ft1k": "30327edcf1390d10e9a0de42a2d731e3",
        "21k": "7970f913eec1b4918e007c8580726412",
        "imagenet": "2b65f5789f4d2f1bf66ecd6d9c5c2d46",
    },
    "m": {
        "21k-ft1k": "0c236c3020e3857de1e5f2939abd0cc6",
        "21k": "3923c286366b2a5137f39d1e5b14e202",
        "imagenet": "ac3fd0ff91b35d18d1df8f1895efe1d5",
    },
    "s": {
        "21k-ft1k": "93046a0d601da46bfce9d4ca14224c83",
        "21k": "10b05d878b64f796ab984a5316a4a1c3",
        "imagenet": "3b91df2c50c7a56071cca428d53b8c0d",
    },
    "t": {"imagenet": "4a0ff9cb396665734d7ca590fa29681b"},
    "xl": {
        "21k-ft1k": "9aaa2bd3c9495b23357bc6593eee5bce",
        "21k": "c97de2770f55701f788644336181e8ee",
    },
    "v1-b0": {
        "noisy_student": "d125a518737c601f8595937219243432",
        "imagenet": "cc7d08887de9df8082da44ce40761986",
    },
    "v1-b1": {
        "noisy_student": "8f44bff58fc5ef99baa3f163b3f5c5e8",
        "imagenet": "a967f7be55a0125c898d650502c0cfd0",
    },
    "v1-b2": {
        "noisy_student": "b4ffed8b9262df4facc5e20557983ef8",
        "imagenet": "6c8d1d3699275c7d1867d08e219e00a7",
    },
    "v1-b3": {
        "noisy_student": "9d696365378a1ebf987d0e46a9d26ddd",
        "imagenet": "d78edb3dc7007721eda781c04bd4af62",
    },
    "v1-b4": {
        "noisy_student": "a0f61b977544493e6926186463d26294",
        "imagenet": "4c83aa5c86d58746a56675565d4f2051",
    },
    "v1-b5": {
        "noisy_student": "c3b6eb3f1f7a1e9de6d9a93e474455b1",
        "imagenet": "0bda50943b8e8d0fadcbad82c17c40f5",
    },
    "v1-b6": {
        "noisy_student": "20dd18b0df60cd7c0387c8af47bd96f8",
        "imagenet": "da13735af8209f675d7d7d03a54bfa27",
    },
    "v1-b7": {
        "noisy_student": "7f6f6dd4e8105e32432607ad28cfad0f",
        "imagenet": "d9c22b5b030d1e4f4c3a96dbf5f21ce6",
    },
    "v1-l2": {"noisy_student": "5fedc721febfca4b08b03d1f18a4a3ca"},
}


def _make_divisible(v, divisor=4, min_value=None):
    """
    This function is taken from the original tf repo.
    It ensures that all layers have a channel number that is divisible by 8
    It can be seen here:
    https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet/mobilenet.py
    """
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v


def conv2d_no_bias(
    inputs,
    filters,
    kernel_size,
    strides=1,
    padding="VALID",
    use_torch_padding=False,
    name="",
):
    pad = (
        max(kernel_size) // 2
        if isinstance(kernel_size, (list, tuple))
        else kernel_size // 2
    )
    if use_torch_padding and padding.upper() == "SAME" and pad != 0:
        inputs = keras.layers.ZeroPadding2D(padding=pad, name=name and name + "pad")(
            inputs
        )
        padding = "VALID"

    return Conv2D(
        filters,
        kernel_size,
        strides=strides,
        padding=padding,
        use_bias=False,
        kernel_initializer=CONV_KERNEL_INITIALIZER,
        name=name + "conv",
    )(inputs)


def batchnorm_with_activation(inputs, activation="Mish", use_torch_eps=False, name=""):
    """Performs a batch normalization followed by an activation."""
    bn_axis = 1 if K.image_data_format() == "channels_first" else -1
    nn = BatchNormalization(
        axis=bn_axis,
        momentum=BATCH_NORM_DECAY,
        epsilon=TORCH_BATCH_NORM_EPSILON if use_torch_eps else BATCH_NORM_EPSILON,
        name=name + "bn",
    )(inputs)
    if activation:
        nn = Activation(activation=activation, name=name + activation)(nn)
        # nn = PReLU(shared_axes=[1, 2], alpha_initializer=tf.initializers.Constant(0.25), name=name + "PReLU")(nn)
    return nn


def se_module(inputs, se_ratio=4, name=""):
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    h_axis, w_axis = [2, 3] if K.image_data_format() == "channels_first" else [1, 2]

    filters = inputs.shape[channel_axis]
    # reduction = _make_divisible(filters // se_ratio, 8)
    reduction = filters // se_ratio
    # se = GlobalAveragePooling2D()(inputs)
    # se = Reshape((1, 1, filters))(se)
    se = tf.reduce_mean(inputs, [h_axis, w_axis], keepdims=True)
    se = Conv2D(
        reduction,
        kernel_size=1,
        use_bias=True,
        kernel_initializer=CONV_KERNEL_INITIALIZER,
        name=name + "1_conv",
    )(se)
    # se = PReLU(shared_axes=[1, 2])(se)
    se = Activation("Mish")(se)
    se = Conv2D(
        filters,
        kernel_size=1,
        use_bias=True,
        kernel_initializer=CONV_KERNEL_INITIALIZER,
        name=name + "2_conv",
    )(se)
    se = Activation("sigmoid")(se)
    return Multiply()([inputs, se])


def MBConv(
    inputs,
    output_channel,
    stride,
    expand_ratio,
    shortcut,
    kernel_size=3,
    drop_rate=0,
    use_se=0,
    is_fused=False,
    is_torch_mode=False,
    name="",
):
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    input_channel = inputs.shape[channel_axis]

    if is_fused and expand_ratio != 1:
        nn = conv2d_no_bias(
            inputs,
            input_channel * expand_ratio,
            (3, 3),
            stride,
            padding="same",
            use_torch_padding=is_torch_mode,
            name=name + "sortcut_",
        )
        nn = batchnorm_with_activation(
            nn, use_torch_eps=is_torch_mode, name=name + "sortcut_"
        )
    elif expand_ratio != 1:
        nn = conv2d_no_bias(
            inputs,
            input_channel * expand_ratio,
            (1, 1),
            strides=(1, 1),
            padding="valid",
            name=name + "sortcut_",
        )
        nn = batchnorm_with_activation(
            nn, use_torch_eps=is_torch_mode, name=name + "sortcut_"
        )
    else:
        nn = inputs

    if not is_fused:
        if is_torch_mode and kernel_size // 2 > 0:
            nn = keras.layers.ZeroPadding2D(
                padding=kernel_size // 2, name=name + "pad"
            )(nn)
            pad = "VALID"
        else:
            pad = "SAME"
        nn = DepthwiseConv2D(
            kernel_size,
            padding=pad,
            strides=stride,
            use_bias=False,
            depthwise_initializer=CONV_KERNEL_INITIALIZER,
            name=name + "MB_dw_",
        )(nn)
        nn = batchnorm_with_activation(
            nn, use_torch_eps=is_torch_mode, name=name + "MB_dw_"
        )

    if use_se:
        nn = se_module(nn, se_ratio=4 * expand_ratio, name=name + "se_")

    # pw-linear
    if is_fused and expand_ratio == 1:
        nn = conv2d_no_bias(
            nn,
            output_channel,
            (3, 3),
            strides=stride,
            padding="same",
            use_torch_padding=is_torch_mode,
            name=name + "fu_",
        )
        nn = batchnorm_with_activation(
            nn, use_torch_eps=is_torch_mode, name=name + "fu_"
        )
    else:
        nn = conv2d_no_bias(
            nn,
            output_channel,
            (1, 1),
            strides=(1, 1),
            padding="valid",
            name=name + "MB_pw_",
        )
        nn = batchnorm_with_activation(
            nn, use_torch_eps=is_torch_mode, activation=None, name=name + "MB_pw_"
        )

    if shortcut:
        if drop_rate > 0:
            nn = Dropout(drop_rate, noise_shape=(None, 1, 1, 1), name=name + "drop")(nn)
        return Add()([inputs, nn])
    else:
        return nn


def EfficientNetV2(
    model_type,
    input_shape=(None, None, 3),
    num_classes=1000,
    dropout=0.2,
    first_strides=2,
    is_torch_mode=False,
    drop_connect_rate=0,
    classifier_activation="softmax",
    include_preprocessing=False,
    pretrained="imagenet",
    model_name="EfficientNetV2",
    kwargs=None,  # Not used, just recieving parameter
):
    if isinstance(model_type, dict):  # For EfficientNetV1 configures
        model_type, blocks_config = model_type.popitem()
    else:
        blocks_config = BLOCK_CONFIGS.get(model_type.lower(), BLOCK_CONFIGS["s"])
    expands = blocks_config["expands"]
    out_channels = blocks_config["out_channels"]
    depthes = blocks_config["depthes"]
    strides = blocks_config["strides"]
    use_ses = blocks_config["use_ses"]
    first_conv_filter = blocks_config.get("first_conv_filter", out_channels[0])
    output_conv_filter = blocks_config.get("output_conv_filter", 1280)
    kernel_sizes = blocks_config.get("kernel_sizes", [3] * len(depthes))
    # "torch" for all V1 models
    # for V2 models, "21k" pretrained are all "tf", "imagenet" pretrained "bx" models are all "torch", ["s", "m", "l", "xl"] are "tf"
    rescale_mode = (
        "tf"
        if pretrained is not None and pretrained.startswith("imagenet21k")
        else blocks_config.get("rescale_mode", "torch")
    )

    inputs = Input(shape=input_shape)
    if include_preprocessing and rescale_mode == "torch":
        channel_axis = 1 if K.image_data_format() == "channels_first" else -1
        Normalization = (
            keras.layers.Normalization
            if hasattr(keras.layers, "Normalization")
            else keras.layers.experimental.preprocessing.Normalization
        )
        mean = tf.constant([0.485, 0.456, 0.406]) * 255.0
        std = (tf.constant([0.229, 0.224, 0.225]) * 255.0) ** 2
        nn = Normalization(mean=mean, variance=std, axis=channel_axis)(inputs)
    elif include_preprocessing and rescale_mode == "tf":
        Rescaling = (
            keras.layers.Rescaling
            if hasattr(keras.layers, "Rescaling")
            else keras.layers.experimental.preprocessing.Rescaling
        )
        nn = Rescaling(scale=1.0 / 128.0, offset=-1)(inputs)
    else:
        nn = inputs
    out_channel = _make_divisible(first_conv_filter, 8)
    nn = conv2d_no_bias(
        nn,
        out_channel,
        (3, 3),
        strides=first_strides,
        padding="same",
        use_torch_padding=is_torch_mode,
        name="stem_",
    )
    nn = batchnorm_with_activation(nn, use_torch_eps=is_torch_mode, name="stem_")

    pre_out = out_channel
    global_block_id = 0
    total_blocks = sum(depthes)
    for id, (expand, out_channel, depth, stride, se, kernel_size) in enumerate(
        zip(expands, out_channels, depthes, strides, use_ses, kernel_sizes)
    ):
        out = _make_divisible(out_channel, 8)
        is_fused = True if se == 0 else False
        for block_id in range(depth):
            stride = stride if block_id == 0 else 1
            shortcut = True if out == pre_out and stride == 1 else False
            name = "stack_{}_block{}_".format(id, block_id)
            block_drop_rate = drop_connect_rate * global_block_id / total_blocks
            nn = MBConv(
                nn,
                out,
                stride,
                expand,
                shortcut,
                kernel_size,
                block_drop_rate,
                se,
                is_fused,
                is_torch_mode,
                name=name,
            )
            pre_out = out
            global_block_id += 1

    output_conv_filter = _make_divisible(output_conv_filter, 8)
    nn = conv2d_no_bias(
        nn, output_conv_filter, (1, 1), strides=(1, 1), padding="valid", name="post_"
    )
    nn = batchnorm_with_activation(nn, use_torch_eps=is_torch_mode, name="post_")

    if num_classes > 0:
        nn = GlobalAveragePooling2D(name="avg_pool")(nn)
        if dropout > 0 and dropout < 1:
            nn = Dropout(dropout)(nn)
        nn = Dense(
            num_classes,
            activation=classifier_activation,
            dtype="float32",
            name="predictions",
        )(nn)

    model = Model(inputs=inputs, outputs=nn, name=model_name)
    model.rescale_mode = rescale_mode
    reload_model_weights(model, model_type, pretrained)
    return model


def reload_model_weights(model, model_type, pretrained="imagenet"):
    pretrained_dd = {
        "imagenet": "imagenet",
        "imagenet21k": "21k",
        "imagenet21k-ft1k": "21k-ft1k",
        "noisy_student": "noisy_student",
    }
    if not pretrained in pretrained_dd:
        print(">>>> No pretrained available, model will be randomly initialized")
        return
    pre_tt = pretrained_dd[pretrained]
    if model_type not in FILE_HASH_DICT or pre_tt not in FILE_HASH_DICT[model_type]:
        print(">>>> No pretrained available, model will be randomly initialized")
        return

    if model_type.startswith("v1"):
        pre_url = "https://github.com/leondgarse/keras_efficientnet_v2/releases/download/effnetv1_pretrained/efficientnet{}-{}.h5"
    else:
        pre_url = "https://github.com/leondgarse/keras_efficientnet_v2/releases/download/effnetv2_pretrained/efficientnetv2-{}-{}.h5"
    url = pre_url.format(model_type, pre_tt)
    file_name = os.path.basename(url)
    file_hash = FILE_HASH_DICT[model_type][pre_tt]

    try:
        pretrained_model = keras.utils.get_file(
            file_name, url, cache_subdir="models/efficientnetv2", file_hash=file_hash
        )
    except:
        print("[Error] will not load weights, url not found or download failed:", url)
        return
    else:
        print(">>>> Load pretrained from:", pretrained_model)
        model.load_weights(pretrained_model, by_name=True, skip_mismatch=True)


def EfficientNetV2B0(
    input_shape=(224, 224, 3),
    num_classes=1000,
    dropout=0.2,
    classifier_activation="softmax",
    pretrained="imagenet",
    **kwargs
):
    return EfficientNetV2(
        model_type="b0", model_name="EfficientNetV2B0", **locals(), **kwargs
    )


def EfficientNetV2B1(
    input_shape=(240, 240, 3),
    num_classes=1000,
    dropout=0.2,
    classifier_activation="softmax",
    pretrained="imagenet",
    **kwargs
):
    return EfficientNetV2(
        model_type="b1", model_name="EfficientNetV2B1", **locals(), **kwargs
    )


def EfficientNetV2B2(
    input_shape=(260, 260, 3),
    num_classes=1000,
    dropout=0.3,
    classifier_activation="softmax",
    pretrained="imagenet",
    **kwargs
):
    return EfficientNetV2(
        model_type="b2", model_name="EfficientNetV2B2", **locals(), **kwargs
    )


def EfficientNetV2B3(
    input_shape=(300, 300, 3),
    num_classes=1000,
    dropout=0.3,
    classifier_activation="softmax",
    pretrained="imagenet",
    **kwargs
):
    return EfficientNetV2(
        model_type="b3", model_name="EfficientNetV2B3", **locals(), **kwargs
    )


def EfficientNetV2T(
    input_shape=(320, 320, 3),
    num_classes=1000,
    dropout=0.2,
    classifier_activation="softmax",
    pretrained="imagenet",
    **kwargs
):
    is_torch_mode = True
    return EfficientNetV2(
        model_type="t", model_name="EfficientNetV2T", **locals(), **kwargs
    )


def EfficientNetV2S(
    input_shape=(384, 384, 3),
    num_classes=1000,
    dropout=0.2,
    classifier_activation="softmax",
    pretrained="imagenet",
    **kwargs
):
    return EfficientNetV2(
        model_type="s", model_name="EfficientNetV2S", **locals(), **kwargs
    )


def EfficientNetV2M(
    input_shape=(480, 480, 3),
    num_classes=1000,
    dropout=0.3,
    classifier_activation="softmax",
    pretrained="imagenet",
    **kwargs
):
    return EfficientNetV2(
        model_type="m", model_name="EfficientNetV2M", **locals(), **kwargs
    )


def EfficientNetV2L(
    input_shape=(480, 480, 3),
    num_classes=1000,
    dropout=0.4,
    classifier_activation="softmax",
    pretrained="imagenet",
    **kwargs
):
    return EfficientNetV2(
        model_type="l", model_name="EfficientNetV2L", **locals(), **kwargs
    )


def EfficientNetV2XL(
    input_shape=(512, 512, 3),
    num_classes=1000,
    dropout=0.4,
    classifier_activation="softmax",
    pretrained="imagenet21k-ft1k",
    **kwargs
):
    return EfficientNetV2(
        model_type="xl", model_name="EfficientNetV2XL", **locals(), **kwargs
    )


def get_actual_drop_connect_rates(model):
    return [ii.rate for ii in model.layers if isinstance(ii, keras.layers.Dropout)]

In [ ]:
import keras.backend as K
import tensorflow as tf
import numpy as np
from sklearn.metrics import roc_auc_score

# metrics
def auc(y_true, y_pred):
    # First, we need to convert the one-hot encoded labels and predicted probabilities
    # into a single label and probability for each sample.
    y_true = np.argmax(y_true, axis=1)
    y_pred = y_pred[:, 1:]

    # Then we can use the roc_auc_score function to compute the AUC for each class
    auc_scores = []
    for i in range(y_pred.shape[1]):
        auc_scores.append(roc_auc_score(y_true, y_pred[:, i]))

    # Return the average AUC across all classes
    return np.mean(auc_scores)


def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))


def tf_mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.5):
        y_pred_ = tf.cast(y_pred > t, tf.int32)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        prec.append(score)
    val = K.mean(K.stack(prec), axis=0)
    return [val, up_opt]


def cross_entropy_balanced(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)

    count_neg = tf.reduce_sum(1.0 - y_true)
    count_pos = tf.reduce_sum(y_true)

    beta = count_neg / (count_pos + count_neg)

    pos_weight = beta / (1 - beta)

    cost = tf.nn.weighted_cross_entropy_with_logits(
        logits=y_pred, labels=y_true, pos_weight=pos_weight
    )

    cost = tf.reduce_mean(cost * (1 - beta))

    return tf.where(tf.equal(count_pos, 0.0), 0.0, cost)


def specificity(y_true, y_pred):
    """
    Calculates the specificity metric.
    """
    true_negatives = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    true_negatives_and_false_positives = K.sum(K.round(K.clip(1 - y_true, 0, 1)))
    return true_negatives / (true_negatives_and_false_positives + K.epsilon())

In [ ]:
# get model
model = EfficientNetV2B0(num_classes=9, input_shape=(256, 256, 3))

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img

# from tensorflow.keras.utils import array_to_image
import csv
from keras.optimizers import optimizer
from keras.callbacks import (
    ModelCheckpoint,
    EarlyStopping,
    TensorBoard,
    ReduceLROnPlateau,
)
from keras.callbacks import LearningRateScheduler
import matplotlib.pyplot as plt

!pip install git+https://github.com/artemmavrin/focal-loss.git

from focal_loss import SparseCategoricalFocalLoss


# Define the callbacks
callbacks = [
    ModelCheckpoint("best_model.h5", save_best_only=True, monitor="accuracy"),
    EarlyStopping(monitor="accuracy", patience=3),
    TensorBoard(log_dir="logs"),
    ReduceLROnPlateau(monitor="loss", patience=1, factor=0.1),
]


def lerp(start, end, weight):
    return start + weight * (end - start)


def sparse_lerp(start, end, weight):
    # Mathematically equivalent, but you can't subtract a dense Tensor from sparse
    # IndexedSlices, so we have to flip it around.
    return start + weight * -(start - end)

# Lion optimizer
class Lion(optimizer.Optimizer):
    r"""Optimizer that implements the Lion algorithm.
    Lion was published in the paper "Symbolic Discovery of Optimization Algorithms"
    which is available at https://arxiv.org/abs/2302.06675
    Args:
      learning_rate: A `tf.Tensor`, floating point value, a schedule that is a
        `tf.keras.optimizers.schedules.LearningRateSchedule`, or a callable
        that takes no arguments and returns the actual value to use. The
        learning rate. Defaults to 1e-4.
      beta_1: A float value or a constant float tensor, or a callable
        that takes no arguments and returns the actual value to use. Factor
         used to interpolate the current gradient and the momentum. Defaults to 0.9.
      beta_2: A float value or a constant float tensor, or a callable
        that takes no arguments and returns the actual value to use. The
        exponential decay rate for the momentum. Defaults to 0.99.
    Notes:
    The sparse implementation of this algorithm (used when the gradient is an
    IndexedSlices object, typically because of `tf.gather` or an embedding
    lookup in the forward pass) does apply momentum to variable slices even if
    they were not used in the forward pass (meaning they have a gradient equal
    to zero). Momentum decay (beta2) is also applied to the entire momentum
    accumulator. This means that the sparse behavior is equivalent to the dense
    behavior (in contrast to some momentum implementations which ignore momentum
    unless a variable slice was actually used).
    """

    def __init__(
        self,
        learning_rate=1e-4,
        beta_1=0.9,
        beta_2=0.99,
        weight_decay=None,
        clipnorm=None,
        clipvalue=None,
        global_clipnorm=None,
        jit_compile=True,
        name="Lion",
        **kwargs,
    ):
        super().__init__(
            name=name,
            weight_decay=weight_decay,
            clipnorm=clipnorm,
            clipvalue=clipvalue,
            global_clipnorm=global_clipnorm,
            jit_compile=jit_compile,
            **kwargs,
        )
        self._learning_rate = self._build_learning_rate(learning_rate)
        self.beta_1 = beta_1
        self.beta_2 = beta_2

    def build(self, var_list):
        """Initialize optimizer variables.
        var_list: list of model variables to build Lion variables on.
        """
        super().build(var_list)
        if hasattr(self, "_built") and self._built:
            return
        self._built = True
        self._emas = []
        for var in var_list:
            self._emas.append(
                self.add_variable_from_reference(
                    model_variable=var, variable_name="ema"
                )
            )

    def update_step(self, gradient, variable):
        """Update step given gradient and the associated model variable."""
        lr = tf.cast(self.learning_rate, variable.dtype)
        beta_1 = tf.constant(self.beta_1, shape=(1,))
        beta_2 = tf.constant(self.beta_2, shape=(1,))

        var_key = self._var_key(variable)
        ema = self._emas[self._index_dict[var_key]]

        if isinstance(gradient, tf.IndexedSlices):
            # Sparse gradients.
            lerp_fn = sparse_lerp
        else:
            # Dense gradients.
            lerp_fn = lerp

        update = lerp_fn(ema, gradient, 1 - beta_1)
        update = tf.sign(update)
        variable.assign_sub(update * lr)

        ema.assign(lerp_fn(ema, gradient, 1 - beta_2))

    def get_config(self):
        config = super().get_config()

        config.update(
            {
                "learning_rate": self._serialize_hyperparameter(self._learning_rate),
                "beta_1": self.beta_1,
                "beta_2": self.beta_2,
            }
        )
        return config


def focal_loss(gamma=2.0, alpha=4.0):
    gamma = float(gamma)
    alpha = float(alpha)

    def focal_loss_fixed(y_true, y_pred):
        """Focal loss for multi-classification
        FL(p_t)=-alpha(1-p_t)^{gamma}ln(p_t)
        Notice: y_pred is probability after softmax
        gradient is d(Fl)/d(p_t) not d(Fl)/d(x) as described in paper
        d(Fl)/d(p_t) * [p_t(1-p_t)] = d(Fl)/d(x)
        Focal Loss for Dense Object Detection
        https://arxiv.org/abs/1708.02002
        Arguments:
            y_true {tensor} -- ground truth labels, shape of [batch_size, num_cls]
            y_pred {tensor} -- model's output, shape of [batch_size, num_cls]
        Keyword Arguments:
            gamma {float} -- (default: {2.0})
            alpha {float} -- (default: {4.0})
        Returns:
            [tensor] -- loss.
        """
        epsilon = 1.0e-9
        y_true = tf.convert_to_tensor(y_true, tf.float32)
        y_pred = tf.convert_to_tensor(y_pred, tf.float32)

        model_out = tf.add(y_pred, epsilon)
        ce = tf.multiply(y_true, -tf.math.log(model_out))
        weight = tf.multiply(y_true, tf.pow(tf.subtract(1.0, model_out), gamma))
        fl = tf.multiply(alpha, tf.multiply(weight, ce))
        reduced_fl = tf.reduce_max(fl, axis=1)
        return tf.reduce_mean(reduced_fl)

    return focal_loss_fixed


from keras.models import load_model


# load already trained model
def load_m(model_name):
    model = load_model(
        model_name,
        custom_objects={
            "focal_loss_fixed": focal_loss,
            "Lion": Lion,
            "mish": Mish(mish),
            "f1_m": f1_m,
            "precision_m": precision_m,
            "recall_m": recall_m,
        },
    )
    return model


# model = load_m("/content/drive/MyDrive/mod_t_20.04.h5")
model.compile(
    optimizer=Lion(6e-6),
    loss=focal_loss(),
    metrics=[
        "accuracy",
        f1_m,
        precision_m,
        recall_m,
        tf.keras.metrics.SpecificityAtSensitivity(0.5),
    ],
)
# training the model
history = model.fit(
    train_generator,
    epochs=20,
    callbacks=callbacks,
    class_weight=train_class_weights_dict,
)
model.save("trained_model.h5")
# evaluating the model
evals = model.evaluate(test_generator)


def save_eval_results(eval_results, filename):
    # Open the file in write mode and write the evaluation results to it
    with open(filename, "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(model.metrics_names)
        writer.writerow(eval_results)

# plotting the training history as a graph
def plot_history(history, call=None):
    fig, axs = plt.subplots(nrows=1, ncols=len(history.history), figsize=(20, 5))

    for i, metric in enumerate(history.history.keys()):
        axs[i].plot(history.history[metric])
        axs[i].set_title(metric)
        axs[i].set_xlabel("Epoch")
        axs[i].set_ylabel(metric)
    plt.savefig(f"{str(history)}_{call}.png")
    plt.show()

# for further evaluation
def quick_test(test_generator, model, img_size):
    # Get a batch of test data
    batch_x, batch_y = test_generator.__getitem__(0)

    # Generate predictions for the test data
    pred_y = model.predict(batch_x)

    # Create a pred folder if it doesn't exist
    if not os.path.exists("pred"):
        os.makedirs("pred")

    # Loop through the test data and save each image and its predicted label to the pred folder
    for i in range(batch_x.shape[0]):
        # Convert the image array to a PIL image
        img = array_to_img(batch_x[i])

        # Get the predicted label for the image
        pred_label = np.argmax(pred_y[i])

        # Save the image with its predicted label as the filename
        img.save(f"pred/{pred_label}_{i}.jpg")


save_eval_results(evals, "HC_100K_Results")
plot_history(history)
quick_test(test_generator, model, (256, 256))

In [ ]:
#model.save("model.h5")

In [ ]:
# testing other model
model = load_m("/content/best_model.h5")
model.compile(
    optimizer=Lion(6e-6),
    loss=focal_loss(),
    metrics=[
        "accuracy",
        f1_m,
        precision_m,
        recall_m,
        tf.keras.metrics.SpecificityAtSensitivity(0.5),
    ],
)
evals = model.evaluate(test_generator)
save_eval_results(evals, "HC_100K_Results_b")

In [ ]:
model.save("test_train.h5")

In [ ]:
# move models to drive
import shutil

shutil.copy("/content/test_train.h5", "/content/drive/MyDrive/mod_t_26.04_2.h5")
shutil.copy("/content/best_model.h5", "/content/drive/MyDrive/best_mod_b_26.04_2.h5")

In [ ]:
import zipfile
import os

# move other files created during training into zip 
# to download them all at once

def zip_folder(folder_path, zip_path):
    with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zip_file:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zip_file.write(os.path.join(root, file))
    print(f"{folder_path} successfully zipped to {zip_path}")


zip_folder("/content/pred", "/content/pred.zip")